# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2024

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^100 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^100.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 200 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.

# Support function for querying the web API

In [1]:
import numpy as np
import requests as rq

# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1)))==0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://baconbreaker.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']

# Making random queries to the API

In [3]:
challenge_id = 'myUniqueIdentifier49831221' # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
query_results = query(challenge_id, queries)

print(query_results)

[-19.  -7.  -7.  -7. -23. -17. -23.  -3.  -5. -13.  -3.  -1. -19. -17.
  -9. -13. -15.   5. -11.  -5.   9. -11.  -5. -19. -17. -21.   7.   5.
  -3. -17. -27.  -7. -15.  -5. -15.  -5.   3. -17.  13. -11.  -7.  15.
 -13.  -5. -15.  -1.  -1. -13. -13.  -7.  -5. -15.  -1. -19.  -7.  -7.
   1. -15. -11.  23.  -5.  -7. -17. -17. -13.  -5.  -1.  -5.   9.   5.
  -3. -11.  -5.  11. -21.  -5. -11.  -1.   5. -19. -13.   5.   5. -27.
 -15.  -5.   1. -13.  -5.   7. -13.   1. -11.   5.   3. -15.  -7. -17.
 -19. -15.   1. -29.   7.   7.  -1. -19.  -9.  -5.   1. -21.  11.   3.
  -5.  -9. -11.  -9. -21.  -1. -13. -11. -21.   3. -15.   3. -19.  -5.
  -1. -11.  -3.   1.   1. -17. -19. -21.  -7.  -9. -11. -31.   5.  -3.
 -11.  -3.  11.  -3.   3.  13. -25. -19.  17. -17. -21. -11.  -3. -13.
 -17.  13. -13.   5. -35.  -5. -23.   5. -25. -19.  15.  -1.   3. -19.
  -1.  -7.  -1.  -7. -23. -19.  -9.  -9. -15.  -7. -11. -13.   9.  -7.
 -17.  -1. -15.  -3.  -5. -29. -13. -15. -15. -13.  -7.  -7.   7.  11.
   7. 

# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://baconbreaker.pythonanywhere.com/leaderboard/?n=100

**Tip**: to solve a linear program you can use the following import:

```
from scipy.optimize import linprog
```



In [7]:
best_query_number = np.argmax(query_results)
best_query = queries[best_query_number]
best_query_result = query(challenge_id, best_query, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")


Reconstruction attack achieves fraction 0.65 correct values


## My Attack 

In [3]:
from scipy.optimize import linprog

In [5]:
challenge_id = "RandomIDorg6" # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
t = 2*n # number of queries to be asked

queries = np.random.choice([-1,+1], size=(t,n)) # Set of random queries
query_results = query(challenge_id, queries)

c = np.concatenate([np.zeros(n), np.ones(t)]) # Initializing coefficients of decision variables 
                                              # Note that we don't directly minimize on our x values

A_ub = np.zeros((2*t,t+n)) # Initializing the inequality matrix (Our queries) 
                           # Rows: 2 times the number of queries because of the absolute value
                           # Columns: Because we have 100 decision variables and 200 inequality constrains
b_ub = np.zeros(2*t) # Initalizing the inequality constraints(Our query results) 
                     # 2 times the number of queries because of the absolute value

for i in range(t):
    # Representing the constraint Ax-t <= b
    A_ub[i,:n] = queries[i]#Setting the first 100 column values to A our queries 
    A_ub[i,(n+i)] = -1 #Activating the t part of our inequality 
    b_ub[i] = query_results[i] #Activating the b part
    
    # Representing the constraint -Ax-t <= -b
    
    A_ub[i+t,:n] = -queries[i] # Same as above but now its -A 
    A_ub[i+t,(n+i)] = -1 # Same as above still -t
    b_ub[i+t] = -query_results[i] # Same as above but now -b 

bounds = [(-1,1)]*n+[(0,None)]*t
    
res =linprog(c,A_ub = A_ub, b_ub = b_ub, bounds= bounds )

final_res = np.sign(res.x[:100]) # Getting the first 100 values corresponding to our x values 
                                 # Then taking the sign because we want answers in {-1,1}

query(challenge_id, final_res, submit = True)

70